In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import ujson
import json
pd.options.display.max_colwidth = 5000
import matplotlib
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import random
import swifter
import collections
from wordcloud import WordCloud

In [3]:
data = pd.read_feather('../data/feather_files/data2019clean.feather')
data = pd.concat((data,\
        pd.read_feather('../data/feather_files/data2020clean.feather')))
data = data.query("tclass == 'political' or tclass=='nonpolitical'").reset_index(drop=True)
#data.rename(columns={'level_0':'pid'},inplace=True)

In [4]:
if True:
    proto = pd.read_feather('../data/feather_files/proto.feather')
    worddict = pd.read_feather('../data/feather_files/worddict.feather')
    wp_in_u = pd.read_feather('../data/feather_files/wp_in_u.feather')
    proto2 = pd.read_feather('../data/feather_files/proto2.feather')

# Step 2

In [ ]:
#a
pol = data.query("tclass == 'political'").reset_index().text
nonpol = data.query("tclass == 'nonpolitical'").reset_index().text

In [ ]:
long_str_p = ' '.join(pol)
long_str_np = ' '.join(nonpol)
tokens_p = word_tokenize(long_str_p)
tokens_np = word_tokenize(long_str_np) 

In [ ]:
# Exploratory analysis
# word cloud
wordcloud = WordCloud(background_color="white", max_words=5000,\
                  contour_width=6, contour_color='steelblue')
# Generate the first
print("Political wordcloud")
wordcloud.generate(long_str_p)
wordcloud.to_image()

In [ ]:
# Generate the second wordcloud
print("non- Political wordcloud")
wordcloud.generate(long_str_np)
wordcloud.to_image()

In [ ]:
print(f"Number of words are {len(tokens_p)} and {len(tokens_np)}")

counter_p = collections.Counter(tokens_p)
counter_np = collections.Counter(tokens_np)
print(f"Number of distinct words is {len(counter_p)} and {len(counter_np)}")
counter_p.most_common(3)
counter_np.most_common(3)


In [ ]:
counter_p.most_common(15)
counter_np.most_common(15)

In [ ]:
# Make a table of word - #occurances in class 1 - #occurances in class 2
worddict = pd.DataFrame({'word':counter_p.keys(),
                        'pol_occ': counter_p.values()})
worddict['nonpol_occ'] = worddict.word.swifter.apply(lambda x: counter_np.get(x,0))
temp = pd.DataFrame({'word':counter_np.keys(),
                'nonpol_occ': counter_np.values()})
temp['pol_occ'] = temp.word.swifter.apply(lambda x: counter_p.get(x,0))
worddict = pd.concat((worddict,temp)).drop_duplicates('word')\
.sort_values(by='pol_occ',ascending=False)
print(f"Number of distinct words is {len(worddict)}")
# Drop words occuring less than n times
n = 50
worddict = worddict.query('pol_occ >= @n or nonpol_occ >= @n').reset_index(drop=True)
print(f"# words occured at least {n} times is {len(worddict)} wordds")

# Computer score a
worddict['ssum'] = worddict.pol_occ + worddict.nonpol_occ
worddict['sc_n'] = worddict.pol_occ / worddict.ssum
worddict['sc_np'] = worddict.nonpol_occ / worddict.ssum
# Keep top k words in each class
k = 200
temp = worddict.sort_values(by='sc_n', ascending=False)[1:k]
worddict = worddict.sort_values(by='sc_np', ascending=False)[1:k]
worddict = pd.concat((temp, worddict))
worddict = worddict.drop_duplicates('word')\
.sort_values(by='sc_n', ascending=False).reset_index(drop=True)
print(f"Number of words after keeping top {k} words is {len(worddict)}")

In [ ]:
worddict.to_feather('../data/feather_files/worddict.feather')

In [ ]:
proto = pd.DataFrame(columns = worddict.word, index=data.index)
proto.to_feather('../data/feather_files/proto.feather')

In [ ]:
wsam = worddict.word[1]
#.allow_dask_on_strings(enable=True)\
wp_in_u=data.text.swifter\
    .apply(lambda d:( [d.count(wp) for wp in worddict.word]))
wp_in_u = pd.DataFrame(wp_in_u)
wp_in_u.to_feather('../data/feather_files/wp_in_u.feather')

In [7]:
proto2 = pd.DataFrame(proto)
for i in np.arange(len(wp_in_u)):
        proto2.iloc[i] = wp_in_u.iloc[i][0]
proto2.to_feather('../data/feather_files/proto2.feather')

In [12]:
proto2

kellyanne aipac gretchen newsmax hickenlooper cpac fisa napolitano  \
0              0     0        0       0            0    0    0          0   
1              0     0        0       0            0    0    0          0   
2              0     0        0       0            0    0    0          0   
3              0     0        0       0            0    0    0          0   
4              0     0        0       0            0    0    0          0   
...          ...   ...      ...     ...          ...  ...  ...        ...   
506161         0     0        0       0            0    0    0          0   
506162         0     0        0       0            0    0    0          0   
506163         0     0        0       0            0    0    0          0   
506164         0     0        0       0            0    0    0          0   
506165         0     0        0       0            0    0    0          0   

       sasse mccabe  ... gyn cyst karate piv tummy speculum instinctively  \
0          0      0  ...   0    0      0   0     0        0             0   
1          0      0  ...   0    0      0   0     0        0             0   
2          0      0  ...   0    0      0   0     0        0             0   
3          0      0  ...   0    0      0   0     0        0             0   
4          0      0  ...   0    0      0   0     0        0             0   
...      ...    ...  ...  ..  ...    ...  ..   ...      ...           ...   
506161     0      0  ...   0    0      0   0     0        0             0   
506162     0      0  ...   0    0      0   0     0        0             0   
506163     0      0  ...   0    0      0   0     0        0             0   
506164     0      0  ...   0    0      0   0     0        0             0   
506165     0      0  ...   0    0      0   0     0        0             0   

       pooped stepmom stool  
0           0       0     0  
1           0       0     0  
2           0       0     0  
3           0       0     0  
4           0       0     0  
...       ...     ...   ...  
506161      0       0     0  
506162      0       0     0  
506163      0       0     0  
506164      0       0     0  
506165      0       0     0  

[506166 rows x 398 columns]

398